In [1]:
import numpy as np
import os
import sys
import argparse
import glob
import time
from PIL import Image
from StringIO import StringIO
import caffe
from tqdm import tqdm_notebook as tqdm
from os.path import join as pj
import cv2
import math
import json

In [2]:
filepath_by_idx = {}

ROOT = '/data/junction'
for idx in range(31, 41):
    filenames = []
    for root, dirs, files in os.walk('{}/{}'.format(ROOT, idx)):
        for file in files:
            filenames.append(pj(root, file))
    candidates = [filename for filename in filenames if os.path.splitext(filename)[1] in ['.avi', '.mkv']]
    if len(candidates) != 1:
        print('! {} Shit candidates {} != 1'.format(idx, len(candidates)))
        for filename in filenames:
            print(filename)
    else:
        filepath_by_idx[idx] = candidates[0]

In [3]:
def resize_image(im, sz=(256, 256)):
    if im.mode != "RGB":
        im = im.convert('RGB')
    imr = im.resize(sz)
    fh_im = StringIO()
    imr.save(fh_im, format='JPEG')
    fh_im.seek(0)
    return bytearray(fh_im.read())

def caffe_preprocess_and_compute(pimg, caffe_transformer=None, caffe_net=None, output_layers=None):
    if caffe_net is not None:

        # Grab the default output names if none were requested specifically.
        if output_layers is None:
            output_layers = caffe_net.outputs

        img_data_rs = resize_image(pimg, sz=(256, 256))
        image = caffe.io.load_image(StringIO(img_data_rs))

        H, W, _ = image.shape
        _, _, h, w = caffe_net.blobs['data'].data.shape
        h_off = max((H - h) / 2, 0)
        w_off = max((W - w) / 2, 0)
        crop = image[h_off:h_off + h, w_off:w_off + w, :]
        transformed_image = caffe_transformer.preprocess('data', crop)
        transformed_image.shape = (1,) + transformed_image.shape

        input_name = caffe_net.inputs[0]
        all_outputs = caffe_net.forward_all(blobs=output_layers, **{input_name: transformed_image})

        outputs = all_outputs[output_layers[0]][0].astype(float)
        return outputs
    else:
        return []

In [4]:
pycaffe_dir = '.'
model_def = './nsfw_model/deploy.prototxt'
pretrained_model = './nsfw_model/resnet_50_1by2_nsfw.caffemodel'
nsfw_net = caffe.Net(model_def, pretrained_model, caffe.TEST)

In [5]:
caffe_transformer = caffe.io.Transformer({'data': nsfw_net.blobs['data'].data.shape})
caffe_transformer.set_transpose('data', (2, 0, 1))  # move image channels to outermost
caffe_transformer.set_mean('data', np.array([104, 117, 123]))  # subtract the dataset-mean value in each channel
caffe_transformer.set_raw_scale('data', 255)  # rescale from [0, 1] to [0, 255]
caffe_transformer.set_channel_swap('data', (2, 1, 0))  # swap channels from RGB to BGR

In [6]:
def do_it(image_data):
    cv2_im = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    image_data = Image.fromarray(cv2_im)
    scores = caffe_preprocess_and_compute(image_data, caffe_transformer=caffe_transformer, caffe_net=nsfw_net, output_layers=['prob'])
    return scores[1]

In [9]:
def just_do_it_for_filepath(filepath):
    scores = []
    
    cap = cv2.VideoCapture(filepath)
    frame_rate = cap.get(5)  # frame rate
    
    frame_counter = 0
    
    pbar = tqdm(total=120*60)
    while cap.isOpened():
        frame_id = cap.get(1)
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % math.floor(frame_rate) == 0:
            frame_counter += 1

        if frame_id % math.floor(frame_rate * 3) == 0:
            score = do_it(frame)
            scores.append(score)
            
            pbar.update(3)
    pbar.close()

    cap.release()
    return scores

In [10]:
for i, video_file_path in tqdm(list(filepath_by_idx.items())):
    done_idxs = os.listdir('./results')
    name = '{}.json'.format(i)
    if name in done_idxs:
        print('{} is already done'.format(name))
        continue
    
    result = just_do_it_for_filepath(video_file_path)
    with open('./results/' + name, 'w') as fout:
        json.dump(result, fout)

HBox(children=(IntProgress(value=0, max=10), HTML(value=u'')))

HBox(children=(IntProgress(value=0, max=7200), HTML(value=u'')))

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/lib/python2.7/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



33.json is already done


HBox(children=(IntProgress(value=0, max=7200), HTML(value=u'')))

...

HBox(children=(IntProgress(value=0, max=7200), HTML(value=u'')))

HBox(children=(IntProgress(value=0, max=7200), HTML(value=u'')))

HBox(children=(IntProgress(value=0, max=7200), HTML(value=u'')))

HBox(children=(IntProgress(value=0, max=7200), HTML(value=u'')))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

